In [14]:
import random
import numpy as np
import time
import statistics
from collections import namedtuple

# Divide and conquer

## Sorting

In [7]:
n = 10000
randomlist = random.sample(range(0, n), n)
print(randomlist) if len(randomlist) < 20 else print("List too long to print")

List too long to print


### Merge sort

In [16]:
# Average and Worst case performance: O(n log(n))
def mergesort(array):
    if len(array) <= 1:
        return array
    else:
        mid = len(array) // 2
        left = array[:mid]
        right = array[mid:]
        left = mergesort(left)
        right = mergesort(right)
        return merge(left, right)
        

def merge(left, right):
    result = []
    i, j = 0, 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j+= 1
    return result + left[i:] + right[j:]

In [17]:
%%timeit
mergesort(randomlist.copy())

36.9 ms ± 3.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Comparison with bubble sort

In [5]:
# Average and Worst case performance: O(n^2)
def bubble_sort(arr):
    n = len(arr)
    
    for i in range(n - 1):
        # Last i elements are already in place
        for j in range(n - i - 1):
            # Swap if the element found is greater than the next element
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    
    return arr

In [6]:
%%timeit
bubble_sort(randomlist.copy())

3.7 s ± 31.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Insertion Sort

In [18]:
# Average and Worst case performance: O(n^2)
def insertion_sort(arr):
    for i in range(1, len(arr)):
        value = arr[i]
        j = i - 1
        
        while j >= 0 and value < arr[j]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = value
    return arr

In [19]:
%%timeit
insertion_sort(randomlist.copy())

3.29 s ± 281 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
start_time = time.time()
for i in range(7):
    insertion_sort(randomlist.copy())
end_time = time.time()
print(f"Insertion Sort Execution Time: {(end_time - start_time)/7} seconds")

Insertion Sort Execution Time: 3.1421399457114085 seconds


In [45]:
def merge_insertion_sort(array):
    if len(array) <= 30:
        return insertion_sort(array)
    else:
        left = merge_insertion_sort(array[:len(array) // 2])
        right = merge_insertion_sort(array[len(array) // 2:])
        # Merge the two halves
        i = j = k = 0
        while i < len(left) and j < len(right):
            if left[i] <= right[j]:
                array[k] = left[i]
                i += 1
            else:
                array[k] = right[j]
                j += 1
            k += 1

        # Copy any remaining elements
        while i < len(left):
            array[k] = left[i]
            i += 1
            k += 1

        while j < len(right):
            array[k] = right[j]
            j += 1
            k += 1
    return array 

In [49]:
%%timeit
merge_insertion_sort(randomlist.copy())

23 ms ± 434 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Quick Sort

In [25]:
# Average and Worst case performance: O(n log(n))
def partition(array):
    pivot_index = random.randint(len(array)//4, 3*len(array)//4)
    pivot = array[pivot_index]
    array[0], array[pivot_index] = array[pivot_index], array[0]
    
    j = 1
    for i in range(1, len(array)):
        if array[i] < pivot:
            array[i], array[j] = array[j], array[i]
            j += 1
    array[0], array[j-1] = array[j-1], array[0]
    return j-1

def quicksort(array):
    if len(array) <= 1:
        return array
    pivot_index = partition(array)
    return quicksort(array[:pivot_index]) + [array[pivot_index]] + quicksort(array[pivot_index + 1:])

In [40]:
%%timeit
quicksort(randomlist.copy())

29.5 ms ± 3.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%%timeit
sorted(randomlist.copy())

1.6 ms ± 147 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Strassen Matrix Multpilication

In [1]:
# Running time: O(n^2.8)
def strassen_multiply(A, B):
    n = A.shape[0]
    
    # Base case: if the matrices are 64x64 or smaller, use regular matrix multiplication
    if n <= 64:
        return np.dot(A, B)
    
    # Splitting the matrices into quadrants
    mid = n // 2
    A11, A12 = A[:mid, :mid], A[:mid, mid:]
    A21, A22 = A[mid:, :mid], A[mid:, mid:]
    B11, B12 = B[:mid, :mid], B[:mid, mid:]
    B21, B22 = B[mid:, :mid], B[mid:, mid:]
    
    # Recursive steps
    P1 = strassen_multiply(A11 + A22, B11 + B22)
    P2 = strassen_multiply(A21 + A22, B11)
    P3 = strassen_multiply(A11, B12 - B22)
    P4 = strassen_multiply(A22, B21 - B11)
    P5 = strassen_multiply(A11 + A12, B22)
    P6 = strassen_multiply(A21 - A11, B11 + B12)
    P7 = strassen_multiply(A12 - A22, B21 + B22)
    
    # Computing the submatrices of the result matrix
    C11 = P1 + P4 - P5 + P7
    C12 = P3 + P5
    C21 = P2 + P4
    C22 = P1 - P2 + P3 + P6
    
    # Combining the submatrices into a single matrix
    C = np.vstack((np.hstack((C11, C12)), np.hstack((C21, C22))))
    
    return C

In [15]:
def pad_matrix(matrix):
    m, n = matrix.shape

    # Calculate the nearest higher power of 2 for rows and columns
    m_power_of_2 = 2 ** np.ceil(np.log2(m))
    n_power_of_2 = 2 ** np.ceil(np.log2(n))

    # Calculate the number of rows and columns to be added for padding
    rows_to_add = int(m_power_of_2 - m)
    cols_to_add = int(n_power_of_2 - n)

    padded_matrix = np.pad(matrix, ((0, rows_to_add), (0, cols_to_add)), 'constant')
    
    return padded_matrix


In [16]:
rows = 2000
columns = 2000

# Create random matrices
a = np.random.randint(low=1, high=10, size=(rows, columns))
b = np.random.randint(low=1, high=10, size=(rows, columns))

In [17]:
%%timeit
c = pad_matrix(a)
d = pad_matrix(b)
strassen_multiply(c, d)

3.67 s ± 41.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Comparison with naive matrix multiplication

In [18]:
%%timeit
# Running time: O(n^3)
a @ b

28.6 s ± 4.04 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Selection

### Randomized Selection

In [48]:
def partition(array):
    pivot_index = random.randint(len(array)//4, 3*len(array)//4)
    pivot = array[pivot_index]
    array[0], array[pivot_index] = array[pivot_index], array[0]
    
    j = 1
    for i in range(1, len(array)):
        if array[i] < pivot:
            array[i], array[j] = array[j], array[i]
            j += 1
    array[0], array[j-1] = array[j-1], array[0]
    return j-1

def selection(array, order_statistic_i):
    if len(array) < 1:
        return array[0]
    pivot_index = partition(array) + 1
    if order_statistic_i == pivot_index:
        return array[pivot_index - 1]
    elif order_statistic_i < pivot_index:
        return selection(array[:pivot_index], order_statistic_i)
    else:
        return selection(array[pivot_index:], order_statistic_i - pivot_index)

In [66]:
n = 10000000
randomlist = random.sample(range(0, n), n)

In [67]:
%%timeit
selection(randomlist.copy(), n/2+1)

8.59 s ± 1.76 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [68]:
%%timeit
statistics.median(randomlist.copy())

5.72 s ± 95.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Graphs

## Representing graphs

### Undirected graphs

In [30]:
# G = (V, E)
Graph = namedtuple('Graph', ['V', 'E'])

In [31]:
# Konigsberg Graph
V = ['A', 'B', 'C', 'D']
E = [('A', 'B'), ('A', 'B'), ('A', 'C'), ('A', 'C'), ('A', 'D'), ('B', 'D'), ('C', 'D')]
G = Graph(V, E) #This is not the best rapresentation of a graph
G

In [33]:
# Adjacency list representation of an undirected graph
def adjacency_dict(graph):
    adj = {node : [] for node in graph.V}
    for edge in graph.E:
        node1, node2 = edge[0], edge[1]
        adj[node1].append(node2)
        adj[node2].append(node1)
    return adj   

# Adjacency list of Konigsberg graph
adjacency_dict(G)

{'A': ['B', 'B', 'C', 'C', 'D'],
 'B': ['A', 'A', 'D'],
 'C': ['A', 'A', 'D'],
 'D': ['A', 'B', 'C']}

In [34]:
# Adjacency Matrix of an undirected graph
def adjacency_matrix(graph):
    n = len(graph.V)
    adj = np.zeros((n, n))
    for edge in graph.E:
        node1, node2 = edge[0], edge[1]
        adj[node1, node2] += 1
        adj[node2, node1] += 1
    return adj

In [35]:
V = range(4)
E = [(0, 1), (0, 1), (0, 2), (0, 2), (0, 3), (1, 3), (2, 3)]
G = Graph(V, E)

In [36]:
# Adjacency Matrix of Konigsberg graph
adjacency_matrix(G)

array([[0., 2., 2., 1.],
       [2., 0., 0., 1.],
       [2., 0., 0., 1.],
       [1., 1., 1., 0.]])

### Account for Directed graphs

In [38]:
Graph = namedtuple('Graph', ['V', 'E', "is_directed"])

In [39]:
def adjacency_dict(graph):
    adj = {node : [] for node in graph.V}
    for edge in graph.E:
        node1, node2 = edge[0], edge[1]
        adj[node1].append(node2)
        if not graph.is_directed:
            adj[node2].append(node1)
    return adj

In [40]:
def adjacency_matrix(graph):
    n = len(graph.V)
    adj = np.zeros((n, n))
    for edge in graph.E:
        node1, node2 = edge[0], edge[1]
        adj[node1, node2] += 1
        if not graph.is_directed:
            adj[node2, node1] += 1
    return adj

In [46]:
G = Graph(V=range(3),E=[(0, 1), (1, 2), (0, 2)], is_directed=False)

In [48]:
adjacency_dict(G)

array([[0., 1., 1.],
       [1., 0., 1.],
       [1., 1., 0.]])

In [49]:
adjacency_matrix(G)

array([[0., 1., 1.],
       [1., 0., 1.],
       [1., 1., 0.]])

In [53]:
G = Graph(V=range(3),E=[(1, 0), (1, 2), (0, 2)], is_directed=True)

In [54]:
adjacency_dict(G)

{0: [2], 1: [0, 2], 2: []}

In [55]:
adjacency_matrix(G)

array([[0., 0., 1.],
       [1., 0., 1.],
       [0., 0., 0.]])